In [1]:
# --- Cell 1: Kaggle Setup ---
import numpy as np 
import pandas as pd 
import os
print("Files available in /kaggle/input/:")
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# This is where we will unzip your checkpoint
print(f"\nFiles will be unzipped to: /kaggle/working/")

Files available in /kaggle/input/:
/kaggle/input/my-math-model-checkpoint/my_checkpoint/adapter_model.safetensors
/kaggle/input/my-math-model-checkpoint/my_checkpoint/adapter_config.json
/kaggle/input/my-math-model-checkpoint/my_checkpoint/README.md
/kaggle/input/my-math-model-checkpoint/my_checkpoint/tokenizer.json
/kaggle/input/my-math-model-checkpoint/my_checkpoint/tokenizer_config.json
/kaggle/input/my-math-model-checkpoint/my_checkpoint/special_tokens_map.json

Files will be unzipped to: /kaggle/working/


In [2]:
print("Cell 2: Installing unsloth and forcing upgrade on all dependencies...")
!pip install "unsloth[colab-new]"

print("Cell 2 Complete: Unsloth and all dependencies force-upgraded.")
print("!!! IMPORTANT: YOU MUST RESTART THE KERNEL NOW BEFORE RUNNING THE NEXT CELL. !!!")

Cell 2: Installing unsloth and forcing upgrade on all dependencies...
Cell 2 Complete: Unsloth and all dependencies force-upgraded.
!!! IMPORTANT: YOU MUST RESTART THE KERNEL NOW BEFORE RUNNING THE NEXT CELL. !!!


In [3]:
# --- Cell 3: Import All Libraries ---
print("Cell 3: Importing libraries...")
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
from transformers import AutoTokenizer
import pandas as pd
from tqdm import tqdm
import gc
import os
import glob
print("Cell 3 Complete.")

Cell 3: Importing libraries...
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-11-02 17:22:52.080172: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762104172.103905     146 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762104172.111786     146 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
Cell 3 Complete.


In [4]:
# --- Cell 4: Set Model Configuration ---
print("Cell 4: Setting model configuration variables...")
max_seq_length = 1024
dtype = None
load_in_4bit = True
print("Cell 4 Complete.")

Cell 4: Setting model configuration variables...
Cell 4 Complete.


In [5]:
# --- Cell 5 & 6 ---
print("Defining the path to your pre-trained model...")

CHECKPOINT_PATH = "/kaggle/input/my-math-model-checkpoint/my_checkpoint"

if not os.path.exists(CHECKPOINT_PATH):
    print(f"--- 🛑 ERROR: Cannot find folder at {CHECKPOINT_PATH} ---")
else:
    print(f"✅ Checkpoint folder found at: {CHECKPOINT_PATH}")
    print("--- You are ready to run the next cell (Cell 7: Load Model) ---")

Defining the path to your pre-trained model...
✅ Checkpoint folder found at: /kaggle/input/my-math-model-checkpoint/my_checkpoint
--- You are ready to run the next cell (Cell 7: Load Model) ---


In [6]:
# --- Cell 7: Load Your Trained Model ---
print("Cell 7: Loading trained model from checkpoint...")

if os.path.exists(CHECKPOINT_PATH):
    # --- Load the saved model --
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = CHECKPOINT_PATH,         # Load from the path you defined in Cell 6
        max_seq_length = max_seq_length,      # From Cell 4
        dtype = dtype,                        # From Cell 4
        load_in_4bit = load_in_4bit,          # From Cell 4
    )

    # --- Prepare for Inference ---
    FastLanguageModel.for_inference(model)

    print("✅ Model loaded from checkpoint and prepared for inference.")
else:
    print(f"--- 🛑 STOPPING: Cannot load model. Checkpoint path not found. ---")

print("Cell 7 Complete.")

Cell 7: Loading trained model from checkpoint...
==((====))==  Unsloth 2025.10.12: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

Unsloth 2025.10.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


✅ Model loaded from checkpoint and prepared for inference.
Cell 7 Complete.


In [7]:
# --- Cell 8: Generate Predictions ---
print("Cell 8: Generating predictions...")

# --- 1. Load the Test Dataset ---
test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")

# --- 2. Define the Inference Prompt Template ---
inference_prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a meticulous math solution verifier. Your task is to carefully analyze the 'Provided Solution' step-by-step to determine if it logically and correctly reaches the 'Expected Answer' for the given 'Question'. Respond ONLY with 'True' if the solution's reasoning AND final result are correct and match the expected answer, otherwise respond ONLY with 'False'.<|eot_id|><|start_header_id|>user<|end_header_id|>
Question:
{}

Expected Answer:
{}

Provided Solution:
{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

# --- 3. Define an Output Parser ---
def parse_output(response_text):
    clean_output = response_text.strip().lower()

    if clean_output.startswith('true'):
        return True
    elif clean_output.startswith('false'):
        return False
    else:
        # This will catch any unexpected "Let's solve..." answers
        print(f"Warning: Unexpected output format: '{response_text[:50]}...' - Defaulting to False")
        return False

# --- 4. Prediction Loop ---
predictions = []
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Generating predictions on the test set using device: {device}...")
for example in tqdm(test_dataset):
    question = example["question"]
    answer = str(example["answer"])
    solution = str(example["solution"])
    
    prompt = inference_prompt_template.format(question, answer, solution) 

    inputs = tokenizer([prompt], return_tensors = "pt", truncation=True, max_length=max_seq_length).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs, 
            max_new_tokens = 8, 
            use_cache = True, 
            pad_token_id=tokenizer.eos_token_id
        )
    input_length = inputs.input_ids.shape[1]
    generated_token_ids = outputs[0][input_length:]
    response_text = tokenizer.decode(generated_token_ids, skip_special_tokens=True)
    prediction = parse_output(response_text)
    predictions.append(prediction)

print("✅ All predictions generated.")
print("Cell 8 Complete.")

Cell 8: Generating predictions...


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/3.65M [00:00<?, ?B/s]

...

 92%|█████████▏| 9215/10000 [2:40:45<16:01,  1.22s/it]

 97%|█████████▋| 9674/10000 [2:48:46<06:55,  1.28s/it]

 98%|█████████▊| 9766/10000 [2:50:26<05:01,  1.29s/it]

100%|██████████| 10000/10000 [2:54:32<00:00,  1.05s/it]

✅ All predictions generated.
Cell 8 Complete.


In [8]:
# --- Cell 9: Create and Save Submission File ---
print("Cell 9: Creating submission file...")

# --- 1. Create a Pandas DataFrame ---
submission = pd.DataFrame({
    'ID': range(len(predictions)),
    'is_correct': predictions
})

# --- 2. Save to CSV ---
submission_path = 'submission.csv'
submission.to_csv(submission_path, index=False)

print(f"\n🎉 Submission file '{submission_path}' created successfully!")
print("--- You can now download this file from '/kaggle/working/' and submit it. ---")

# --- 3. Display the first 5 rows ---
print("\nFirst 5 predictions:")
print(submission.head())
print(f"\nTotal predictions: {len(submission)}")
print(f"Count of True predictions: {submission['is_correct'].sum()}")
print(f"Count of False predictions: {len(submission) - submission['is_correct'].sum()}")
print("Cell 9 Complete.")

Cell 9: Creating submission file...

🎉 Submission file 'submission.csv' created successfully!
--- You can now download this file from '/kaggle/working/' and submit it. ---

First 5 predictions:
   ID  is_correct
0   0        True
1   1       False
2   2       False
3   3       False
4   4       False

Total predictions: 10000
Count of True predictions: 3611
Count of False predictions: 6389
Cell 9 Complete.
